<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/project/extractFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

import datetime
import collections
import warnings
import csv
import math
import numpy
import statistics
from collections import deque
from math import sqrt
from datetime import datetime

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

warnings.filterwarnings("ignore")

In [0]:
def change_class_name(a):
    if (a in "Colazione, Pranzo, Cena"):
        a = "Pasto"
    if (a == "None"):
        a = "Other"

    return (a)
  
def print_weka_header(feature_vector, class_set):
    s = ""
    i = 1
    my_list = list(class_set)

    if (my_list[0] == "Pasto" and my_list[1] == "Other"):
        pass
    else:
        my_list[0] = "Pasto"
        my_list[1] = "Other"

    s += "@relation uHoo\n"
    for f in feature_vector:
        if (f != "class"):
            s += "@attribute " + str(i) + "-" + str(f) + " " + "numeric\n"
            i += 1
            
    s += "@attribute class" + " {"
    p = 1
    for c in my_list:
        s += c
        if (p < len(class_set)): s += ","
        p += 1
    s += "}\n\n"
    return (s)

  
def print_weka_data(feature_vectors_list):
    s = ""
    s += "@data\n"
    for fv in feature_vectors_list:
        p = 1
        for f in fv:
            s += str(fv[f])
            if (p < len(fv)): s += ","
            p += 1
        s += "\n"
    return (s)

In [0]:
def build_feature_vector(deq_datum, name_datum):
  
    features = collections.OrderedDict({})
    name_datum = str(name_datum)
    window = int(deq_datum.__len__()/2)

    if (name_datum == "datetime"):

        #central minute value
        features["datetime"] = list(deq_datum)[window]

        #max last 5 min
        features["max_" + name_datum + "_last5min"] = max(list(deq_datum)[-(window+5):-window])
        #min last 5 min
        features["min_" + name_datum + "_last5min"] = min(list(deq_datum)[-(window+5):-window])

        #max next 5 min
        features["max_" + name_datum + "_next5min"] = max(list(deq_datum)[-window:-(window-5)])
        #min next 5 min
        features["min_" + name_datum + "_next5min"] = min(list(deq_datum)[-window:-(window-5)])

    else:

        #max last 5 min
        features["max_" + name_datum + "_last5min"] = max(list(deq_datum)[-(window+5):-window])
        #min last 5 min
        features["min_" + name_datum + "_last5min"] = min(list(deq_datum)[-(window+5):-window])
        #mean last 5 min
        features["avg_" + name_datum + "_last5min"] = statistics.mean(list(deq_datum)[-(window+5):-window])
        #standard deviation last 5 min
        features["stdev_" + name_datum + "_last5min"] = statistics.stdev(list(deq_datum)[-(window+5):-window])

        #max next 5 min
        features["max_" + name_datum + "_next5min"] = max(list(deq_datum)[-window:-(window-5)])
        #min next 5 min
        features["min_" + name_datum + "_next5min"] = min(list(deq_datum)[-window:-(window-5)])
        #mean next 5 min
        features["avg_" + name_datum + "_next5min"] = statistics.mean(list(deq_datum)[-window:-(window-5)])
        #standard deviation next 5 min
        features["stdev_" + name_datum + "_next5min"] = statistics.stdev(list(deq_datum)[-window:-(window-5)])
                                                                         
        features["diff_" + name_datum + "_5min"] = deq_datum[-window] - deq_datum[-(window+5)]
        features["diff_" + name_datum + "_10min"] = deq_datum[-window] - deq_datum[-(window+10)]
        features["diff_" + name_datum + "_15min"] = deq_datum[-window] - deq_datum[-(window+15)]
        features["diff_" + name_datum + "_20min"] = deq_datum[-window] - deq_datum[-(window+20)]
        features["diff_" + name_datum + "_25min"] = deq_datum[-window] - deq_datum[-(window+25)]

        features["s_diff_" + name_datum + "_5min"] = deq_datum[-(window-5)] - deq_datum[-window]
        features["s_diff_" + name_datum + "_10min"] = deq_datum[-(window-10)] - deq_datum[-window]
        features["s_diff_" + name_datum + "_15min"] = deq_datum[-(window-15)] - deq_datum[-window]
        features["s_diff_" + name_datum + "_20min"] = deq_datum[-(window-20)] - deq_datum[-window]
        features["s_diff_" + name_datum + "_25min"] = deq_datum[-(window-25)] - deq_datum[-window]

    return (features)

In [0]:
#FUNZIONE PER ESTRARRE SOLO I VALORI CENTRALI (REDUCED)
'''
def build_feature_vector(deq_datum, name_datum):
  
    features = collections.OrderedDict({})
    name_datum = str(name_datum)
    window = int(deq_datum.__len__()/2)
   
    features[name_datum] = list(deq_datum)[window]
      
    return (features)
'''

In [0]:
#FUNZIONE PER ESTRARRE LE FEATURE SOLO DAL PASSATO (SEGMENT_LENGHT CONSIGLIATO 31)
'''
def build_feature_vector(deq_datum, name_datum):
  
    features = collections.OrderedDict({})
    name_datum = str(name_datum)
    minute = -1

    if (name_datum == "datetime"):

        #central minute value
        features["datetime"] = list(deq_datum)[minute]

        #max last 5 min
        features["max_" + name_datum + "_last5min"] = max(list(deq_datum)[(minute-5):minute])
        #min last 5 min
        features["min_" + name_datum + "_last5min"] = min(list(deq_datum)[(minute-5):minute])


    else:

        #max last 5 min
        features["max_" + name_datum + "_last5min"] = max(list(deq_datum)[(minute-5):minute])
        #min last 5 min
        features["min_" + name_datum + "_last5min"] = min(list(deq_datum)[(minute-5):minute])
        #mean last 5 min
        features["avg_" + name_datum + "_last5min"] = statistics.mean(list(deq_datum)[(minute-5):minute])
        #standard deviation last 5 min
        features["stdev_" + name_datum + "_last5min"] = statistics.stdev(list(deq_datum)[(minute-5):minute])

                                                                         
        features["diff_" + name_datum + "_5min"] = deq_datum[minute] - deq_datum[(minute-5)]
        features["diff_" + name_datum + "_10min"] = deq_datum[minute] - deq_datum[(minute-10)]
        features["diff_" + name_datum + "_15min"] = deq_datum[minute] - deq_datum[(minute-15)]
        features["diff_" + name_datum + "_20min"] = deq_datum[minute] - deq_datum[(minute-20)]
        features["diff_" + name_datum + "_25min"] = deq_datum[minute] - deq_datum[(minute-25)]

    return (features)
'''

In [0]:
#@title Scegli il parametro segment_length:  

def extract_features(out_file):

    line_num = 0
    segment_length = 61 #@param {type:"integer"}
    class_set = set([])
    feature_vectors_list = list([])

    for i in range(lista.__len__()):
        if(lista[i] > attributes.__len__()):
            print("Index Error")
            exit()

    if((lista.__len__() > attributes.__len__()) or (lista.__len__() == 0)):
        print("Index Error")
    else:

        for i in range(lista.__len__()):
            if(attributes[lista[i]].__eq__("temperature")):
                queue_temperature = deque([])
            elif (attributes[lista[i]].__eq__("humidity")):
                queue_humidity = deque([])
            elif(attributes[lista[i]].__eq__("co2")):
                queue_co2 = deque([])
            elif (attributes[lista[i]].__eq__("pm25")):
                queue_pm25 = deque([])
            elif (attributes[lista[i]].__eq__("tvoc")):
                queue_tvoc = deque([])
            elif (attributes[lista[i]].__eq__("ozone")):
                queue_ozone = deque([])
            elif (attributes[lista[i]].__eq__("no2")):
                queue_no2 = deque([])
            elif (attributes[lista[i]].__eq__("airpressure")):
                queue_airpressure = deque([])
            elif (attributes[lista[i]].__eq__("datetime")):
                queue_datetime = deque([])
            elif (attributes[lista[i]].__eq__("co")):
                queue_co = deque([])
            elif (attributes[lista[i]].__eq__("classe")):
                queue_classe = deque([])


        with open(in_file, "r") as f:
          
            for line in f:
              
                
                s = line.split(",")
                
                if(s[0][0:4] == 'Date'): continue
                  
                line_num += 1

                timestamp = datetime.strptime(s[0], "%Y-%m-%d %H:%M")
                minute = timestamp.minute
                hour = timestamp.hour
                past_midnight = (hour*60)+minute

                temperature = float(s[1])
                humidity = float(s[2])
                pm25 = float(s[3])
                tvoc = float(s[4])
                co2 = float(s[5])
                co = float(s[6])
                airpressure = float(s[7])
                ozone = float(s[8])
                no2 = float(s[9])
                act = str(s[10])
                
                classe = change_class_name(act)

                  
                for i in range(lista.__len__()):
                        if (attributes[lista[i]].__eq__("temperature")):
                            queue_temperature.append(temperature)
                        elif (attributes[lista[i]].__eq__("humidity")):
                            queue_humidity.append(humidity)
                        elif (attributes[lista[i]].__eq__("co2")):
                            queue_co2.append(co2)
                        elif (attributes[lista[i]].__eq__("pm25")):
                            queue_pm25.append(pm25)
                        elif (attributes[lista[i]].__eq__("tvoc")):
                            queue_tvoc.append(tvoc)
                        elif (attributes[lista[i]].__eq__("ozone")):
                            queue_ozone.append(ozone)
                        elif (attributes[lista[i]].__eq__("no2")):
                            queue_no2.append(no2)
                        elif (attributes[lista[i]].__eq__("airpressure")):
                            queue_airpressure.append(airpressure)
                        elif (attributes[lista[i]].__eq__("datetime")):
                            queue_datetime.append(past_midnight)
                        elif (attributes[lista[i]].__eq__("co")):
                            queue_co.append(co)
                        elif (attributes[lista[i]].__eq__("classe")):
                            queue_classe.append(classe)

                if(line_num > segment_length):

                    if (attributes[lista[0]].__eq__("temperature")):
                        feature_vector = build_feature_vector(queue_temperature, attributes[lista[0]])
                    elif (attributes[lista[0]].__eq__("humidity")):
                        feature_vector = build_feature_vector(queue_humidity, attributes[lista[0]])
                    elif (attributes[lista[0]].__eq__("co2")):
                        feature_vector = build_feature_vector(queue_co2, attributes[lista[0]])
                    elif (attributes[lista[0]].__eq__("pm25")):
                        feature_vector = build_feature_vector(queue_pm25, attributes[lista[0]])
                    elif (attributes[lista[0]].__eq__("tvoc")):
                        feature_vector = build_feature_vector(queue_tvoc, attributes[lista[0]])
                    elif (attributes[lista[0]].__eq__("ozone")):
                        feature_vector = build_feature_vector(queue_ozone, attributes[lista[0]])
                    elif (attributes[lista[0]].__eq__("no2")):
                        feature_vector = build_feature_vector(queue_no2, attributes[lista[0]])
                    elif (attributes[lista[0]].__eq__("airpressure")):
                        feature_vector = build_feature_vector(queue_airpressure, attributes[lista[0]])
                    elif (attributes[lista[0]].__eq__("datetime")):
                        feature_vector = build_feature_vector(queue_datetime, attributes[lista[0]])
                    elif (attributes[lista[0]].__eq__("co")):
                        feature_vector = build_feature_vector(queue_co, attributes[lista[0]])

                    if(lista.__len__() > 1):
                        for i in range(lista.__len__()):
                            if (attributes[lista[i]].__eq__("temperature")):
                                feature_vector.update(build_feature_vector(queue_temperature, attributes[lista[i]]))
                            elif (attributes[lista[i]].__eq__("humidity")):
                                feature_vector.update(build_feature_vector(queue_humidity, attributes[lista[i]]))
                            elif (attributes[lista[i]].__eq__("co2")):
                                feature_vector.update(build_feature_vector(queue_co2, attributes[lista[i]]))
                            elif (attributes[lista[i]].__eq__("pm25")):
                                feature_vector.update(build_feature_vector(queue_pm25, attributes[lista[i]]))
                            elif (attributes[lista[i]].__eq__("tvoc")):
                                feature_vector.update(build_feature_vector(queue_tvoc, attributes[lista[i]]))
                            elif (attributes[lista[i]].__eq__("ozone")):
                                feature_vector.update(build_feature_vector(queue_ozone, attributes[lista[i]]))
                            elif (attributes[lista[i]].__eq__("no2")):
                                feature_vector.update(build_feature_vector(queue_no2, attributes[lista[i]]))
                            elif (attributes[lista[i]].__eq__("airpressure")):
                                feature_vector.update(build_feature_vector(queue_airpressure, attributes[lista[i]]))
                            elif (attributes[lista[i]].__eq__("datetime")):
                                feature_vector.update(build_feature_vector(queue_datetime, attributes[lista[i]]))
                            elif (attributes[lista[i]].__eq__("co")):
                                feature_vector.update(build_feature_vector(queue_co, attributes[lista[i]]))
                        
                        feature_vector["class"] = queue_classe[int(segment_length/2)+1]
                        feature_vectors_list.append(feature_vector)
                        class_set.add(classe)

                    for i in range(lista.__len__()):
                        if (attributes[lista[i]].__eq__("temperature")):
                            queue_temperature.popleft()
                        elif (attributes[lista[i]].__eq__("humidity")):
                            queue_humidity.popleft()
                        elif (attributes[lista[i]].__eq__("co2")):
                            queue_co2.popleft()
                        elif (attributes[lista[i]].__eq__("pm25")):
                            queue_pm25.popleft()
                        elif (attributes[lista[i]].__eq__("tvoc")):
                            queue_tvoc.popleft()
                        elif (attributes[lista[i]].__eq__("ozone")):
                            queue_ozone.popleft()
                        elif (attributes[lista[i]].__eq__("no2")):
                            queue_no2.popleft()
                        elif (attributes[lista[i]].__eq__("airpressure")):
                            queue_airpressure.popleft()
                        elif (attributes[lista[i]].__eq__("datetime")):
                            queue_datetime.popleft()
                        elif (attributes[lista[i]].__eq__("co")):
                            queue_co.popleft()
                        elif (attributes[lista[i]].__eq__("classe")):
                            queue_classe.popleft()
                            
                      

        with open(out_file, "w") as f:
            s = print_weka_header(feature_vectors_list[0], class_set)
            f.write(s)
            s = print_weka_data(feature_vectors_list)
            f.write(s)
            print("Written to: " + str(out_file))
            
        


In [0]:
#@title Scegli il parametro out_file:

in_file = '/root/data/uHoo_Complete_Dataset.csv'
out_file = '/root/data/uHooComplete_featureDataset.arff' #@param {type:"string"}


attributes = ["temperature","humidity","co2","pm25", "tvoc","ozone", "no2", "airpressure", "datetime", "co", "classe"]

print("\nEstrazione delle feature... \nScegli gli attributi che vuoi inserire selezionando il numero della lista (i numeri devono essere separati da uno spazio): ")
print("\n0)Temperature \n1)Humidity \n2)CO2 \n3)PM2.5 \n4)TVOC \n5)Ozone \n6)NO2 \n7)Air Pressure \n8)DateTime \n9)CO \n10)Activity")
print("\nEffettua la scelta: ")

lista = [int(x) for x in input().split()]

extract_features(out_file)

print("Features extracted!!!")
  
  

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Create & upload a file text file.
uploaded = drive.CreateFile()
uploaded.SetContentFile(out_file)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))
